In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:44:15.132953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:44:15.245763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:44:15.245783: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:44:15.267379: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:44:15.808434: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 12:44:20.244431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:44:20.246163: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:44:20.246301: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:44:20.246422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:44:20.246542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 399ms/step - loss: 2.1590 - auc: 0.4669
Epoch 2/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6417 - auc: 0.8262
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7845 - auc: 0.8374
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7884 - auc: 0.8563
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6799 - auc: 0.8719
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5334 - auc: 0.8837
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3952 - auc: 0.8869
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2702 - auc: 0.8958
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1903 - auc: 0.9060
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2048 - auc: 0.8781
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1514 

1/1 [==============================] - 0s 29ms/step - loss: 0.2392 - auc: 0.9902
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2331 - auc: 0.9915
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2328 - auc: 0.9913
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2306 - auc: 0.9914
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2381 - auc: 0.9898
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2358 - auc: 0.9898
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2290 - auc: 0.9919
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2256 - auc: 0.9920
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2263 - auc: 0.9921
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2338 - auc: 0.9902
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 1.9572 - auc: 0.5055
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5991 - auc: 0.8028
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6629 - auc: 0.8322
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6437 - auc: 0.8488
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5245 - auc: 0.8606
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4148 - auc: 0.8612
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2515 - auc: 0.8755
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1803 - auc: 0.8750
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1819 - auc: 0.8510
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1643 - auc: 0.8448
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.2786 - auc: 0.9847
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2601 - auc: 0.9884
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2747 - auc: 0.9854
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2726 - auc: 0.9860
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2723 - auc: 0.9861
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2700 - auc: 0.9865
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2590 - auc: 0.9893
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2660 - auc: 0.9871
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2647 - auc: 0.9871
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2599 - auc: 0.9884
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: 1.6490 - auc: 0.7368
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7198 - auc: 0.8135
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5355 - auc: 0.8250
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3471 - auc: 0.8393
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3009 - auc: 0.8171
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 1.2255 - auc: 0.8409
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0969 - auc: 0.8985
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0116 - auc: 0.9288
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 0.9788 - auc: 0.9363
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 0.9372 - auc: 0.9445
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2216 - auc: 0.9928
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2285 - auc: 0.9911
Epoch 90/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2292 - auc: 0.9914
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2270 - auc: 0.9919
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2310 - auc: 0.9911
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2288 - auc: 0.9915
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2294 - auc: 0.9917
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2274 - auc: 0.9915
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2279 - auc: 0.9920
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2207 - auc: 0.9926
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 2.3721 - auc: 0.3534
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5910 - auc: 0.7601
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5908 - auc: 0.8447
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6809 - auc: 0.8594
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6768 - auc: 0.8728
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6473 - auc: 0.8753
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5166 - auc: 0.8883
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3652 - auc: 0.8975
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2293 - auc: 0.9048
Epoch 10/100
1/1 [==============================] - 0s 37ms/step - loss: 1.0986 - auc: 0.9177
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2392 - auc: 0.9898
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2366 - auc: 0.9906
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2311 - auc: 0.9913
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2276 - auc: 0.9927
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2330 - auc: 0.9905
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2450 - auc: 0.9882
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2360 - auc: 0.9902
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2305 - auc: 0.9913
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2322 - auc: 0.9911
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2271 - auc: 0.9914
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 1.7635 - auc: 0.7528
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6266 - auc: 0.7088
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4537 - auc: 0.8061
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3728 - auc: 0.8471
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2652 - auc: 0.8711
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1465 - auc: 0.8965
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0950 - auc: 0.9024
Epoch 8/100
1/1 [==============================] - 0s 24ms/step - loss: 1.0226 - auc: 0.9195
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9514 - auc: 0.9398
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.8851 - auc: 0.9554
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2240 - auc: 0.9913
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2203 - auc: 0.9926
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2257 - auc: 0.9919
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2303 - auc: 0.9904
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2271 - auc: 0.9906
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2279 - auc: 0.9902
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2166 - auc: 0.9926
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2236 - auc: 0.9911
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2196 - auc: 0.9926
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2219 - auc: 0.9915
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 3.3702 - auc: 0.2290
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9629 - auc: 0.5043
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5234 - auc: 0.8115
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6175 - auc: 0.8500
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7710 - auc: 0.8571
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8705 - auc: 0.8565
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8821 - auc: 0.8635
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7947 - auc: 0.8707
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 1.7183 - auc: 0.8742
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6020 - auc: 0.8822
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.3113 - auc: 0.9819
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2978 - auc: 0.9850
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.3081 - auc: 0.9821
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2926 - auc: 0.9858
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.3016 - auc: 0.9830
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2984 - auc: 0.9836
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.3106 - auc: 0.9811
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2859 - auc: 0.9866
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2940 - auc: 0.9851
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2993 - auc: 0.9833
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [9]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 3.0549 - auc: 0.2477
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.7582 - auc: 0.6305
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 1.5040 - auc: 0.8368
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6091 - auc: 0.8651
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7610 - auc: 0.8677
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8039 - auc: 0.8665
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 1.7747 - auc: 0.8753
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6993 - auc: 0.8790
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5827 - auc: 0.8862
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4513 - auc: 0.8893
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2749 - auc: 0.9882
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2747 - auc: 0.9877
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2564 - auc: 0.9907
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2637 - auc: 0.9894
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2631 - auc: 0.9896
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2643 - auc: 0.9898
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2584 - auc: 0.9905
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2641 - auc: 0.9890
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2629 - auc: 0.9890
Epoch 97/100
1/1 [==============================] - 0s 36ms/step - loss: 0.2663 - auc: 0.9879
Epoch 98/100
1/1 [==============================] - 0s 22ms/step - loss: 

In [10]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.7732 - auc: 0.6243
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6274 - auc: 0.8156
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6473 - auc: 0.8387
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5640 - auc: 0.8483
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3736 - auc: 0.8626
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 1.2810 - auc: 0.8563
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2201 - auc: 0.8560
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1994 - auc: 0.8460
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1179 - auc: 0.8744
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0513 - auc: 0.8996
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2511 - auc: 0.9891
Epoch 89/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2526 - auc: 0.9894
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2499 - auc: 0.9898
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2526 - auc: 0.9893
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2607 - auc: 0.9874
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2501 - auc: 0.9905
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2565 - auc: 0.9891
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2538 - auc: 0.9894
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2540 - auc: 0.9884
Epoch 97/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2572 - auc: 0.9876
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [11]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 524ms/step - loss: 1.7415 - auc: 0.7765
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6114 - auc: 0.7246
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4568 - auc: 0.7866
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3536 - auc: 0.8334
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2767 - auc: 0.8657
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1507 - auc: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0575 - auc: 0.9202
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0052 - auc: 0.9287
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9202 - auc: 0.9516
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 0.8874 - auc: 0.9548
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2248 - auc: 0.9912
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2253 - auc: 0.9905
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2267 - auc: 0.9908
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2123 - auc: 0.9936
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2253 - auc: 0.9911
Epoch 93/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2224 - auc: 0.9918
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2192 - auc: 0.9929
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2154 - auc: 0.9925
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2172 - auc: 0.9929
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2135 - auc: 0.9931
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [12]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('8feature_remove_one3.csv',index=False)
print(de)

                 model       auc
0            all_model  0.947608
1    remove_AtomPairFP  0.927965
2  remove_Connectivity  0.948027
3        remove_Charge  0.946668
4         remove_Kappa  0.947918
5          remove_MAP4  0.943926
6           remove_MOE  0.946268
7      remove_MorganFP  0.945767
8      remove_Topology  0.947676
